In [15]:
import json
import torch
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration, Trainer, TrainingArguments
from datasets import Dataset

# ✅ 1️⃣ 학습 데이터 로드
with open("labeling_novels_filtered.json", "r", encoding="utf-8") as f:
    labeled_data = json.load(f)

In [16]:
# ✅ 2️⃣ 데이터셋 변환 (프롬프트 추가)
train_data = [{
    "input": f"다음 문장 중 작가의 연혁과 수상이력은 제외하고 스토리부분만을 요약해주세요. 등장인물의 직업, 스토리 전개, 주요 사건을 포함해주세요. :\n\n{item['description']}",
    "target": item["summary"]
} for item in labeled_data]

dataset = Dataset.from_dict({
    "input": [item["input"] for item in train_data],
    "target": [item["target"] for item in train_data]
})

In [17]:
# ✅ 3️⃣ KoBART 모델 & 토크나이저 로드
model_name = "gogamza/kobart-summarization"
tokenizer = PreTrainedTokenizerFast.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [18]:
# ✅ 4️⃣ 데이터 토크나이징 (Trainer가 이해할 수 있는 형태로 변환)
def preprocess_data(example):
    model_inputs = tokenizer(
        example["input"], 
        max_length=1024, 
        truncation=True, 
        padding="max_length"
    )
    
    # 🔹 "target"을 "labels"로 변환 후 토크나이징
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            example["target"], 
            max_length=250, 
            truncation=True, 
            padding="max_length"
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [19]:
# ✅ 5️⃣ 데이터셋 변환 (batched=True로 속도 최적화)
tokenized_dataset = dataset.map(preprocess_data, batched=True, remove_columns=["input", "target"])

Map:   0%|          | 0/217 [00:00<?, ? examples/s]

/opt/anaconda3/envs/hi/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3970: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [20]:
# ✅ 6️⃣ 학습 설정
training_args = TrainingArguments(
    output_dir="./kobart_summary_finetuned",
    evaluation_strategy="epoch",  # 🔹 매 epoch마다 평가
    save_strategy="epoch",        # 🔹 매 epoch마다 저장
    per_device_train_batch_size=2,  # 🔹 배치 크기 (GPU 메모리에 따라 조정 가능)
    per_device_eval_batch_size=2,
    num_train_epochs=5,  # 🔹 학습 횟수 (필요에 따라 조정 가능)
    weight_decay=0.01,  # 🔹 가중치 감쇠 (과적합 방지)
    save_total_limit=5,  # 🔹 체크포인트 최대 개수
    logging_dir="./logs",
    logging_steps=100
)

/opt/anaconda3/envs/hi/lib/python3.10/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [21]:
# ✅ 7️⃣ 🤗 Datasets의 train_test_split() 사용
dataset_split = tokenized_dataset.train_test_split(test_size=0.2, seed=42)

# ✅ 8️⃣ train/eval 데이터셋 설정
train_data = dataset_split["train"]
eval_data = dataset_split["test"]

# ✅ 9️⃣ Trainer 객체 생성 (eval_dataset 추가)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=eval_data,  # 🔹 평가 데이터셋 추가!
    tokenizer=tokenizer
)


/var/folders/wz/cf10k4857715p70flsxw2g9m0000gn/T/ipykernel_2894/3899536195.py:9: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [22]:
# ✅ 10 학습 시작 🚀
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,0.811702
2,2.022500,0.783771
3,0.513200,0.811638
4,0.297200,0.836755
5,0.213300,0.850627


TrainOutput(global_step=435, training_loss=0.7107568982003749, metrics={'train_runtime': 191.9181, 'train_samples_per_second': 4.507, 'train_steps_per_second': 2.267, 'total_flos': 527422036377600.0, 'train_loss': 0.7107568982003749, 'epoch': 5.0})

In [10]:
# ✅ 9️⃣ 모델 저장
model.save_pretrained("./kobart_summary_finetuned")
tokenizer.save_pretrained("./kobart_summary_finetuned")

print("✅ KoBART 파인튜닝 완료! 모델이 './kobart_summary_finetuned'에 저장되었습니다.")

✅ KoBART 파인튜닝 완료! 모델이 './kobart_summary_finetuned'에 저장되었습니다.


In [24]:
from transformers import BartForConditionalGeneration, PreTrainedTokenizerFast

best_checkpoint = "./kobart_summary_finetuned/checkpoint-174"  # epoch 2 모델 체크포인트
save_path = "./kobart_best_model"

# 모델과 토크나이저 불러오기
model = BartForConditionalGeneration.from_pretrained(best_checkpoint)
tokenizer = PreTrainedTokenizerFast.from_pretrained(best_checkpoint)

# 저장
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print(f"✅ 가장 좋은 모델이 '{save_path}'에 저장되었습니다!")

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


✅ 가장 좋은 모델이 './kobart_best_model'에 저장되었습니다!


In [26]:
from transformers import BartForConditionalGeneration, PreTrainedTokenizerFast

# ✅ 저장된 모델 불러오기
best_checkpoint = "./kobart_best_model"
model = BartForConditionalGeneration.from_pretrained(best_checkpoint)
tokenizer = PreTrainedTokenizerFast.from_pretrained(best_checkpoint)

# ✅ 테스트할 입력 문장
test_text = "독자를 공모자로 끌어들인 채 관찰하는 자매의 하룻밤!\n\n무라카미 하루키 데뷔 25주년 기념 장편소설 『애프터 다크』.\n 1979년 《바람의 노래를 들어라》로 군조신인문학상을 수상하며 데뷔한 이후 독자들의 사랑을 받아온 무라카미 하루키.\n 그가 등단 25주년을 맞는 해에 발표한 11번째 장편소설로, 발표 시기적으로는 《해변의 카프카》와 《1Q84》 사이에, 볼륨으로는 《국경의 남쪽, 태양의 서쪽》, 《스푸트니크의 연인》과 같은 장편소설 옆에 나란히 위치한다.\n\n\n작품은 자정이 가까운 한밤에서부터 새날이 밝아오는 아침까지 일곱 시간 동안 벌어지는 백설 공주처럼 예쁜 언니 ‘에리’와 씩씩한 양치기 목동 같은 동생 ‘마리’, 두 자매의 이야기를 담고 있다.\n ‘우리’라고 명명된 카메라의 시선이 이야기를 주도한다.\n 높은 곳에서 조감하는가 하면, 때로는 근접하여 클로즈업을 시도하며 영화의 장면들처럼 에리의 밤과 마리의 밤을 교대로 비추는 동안 작가는 어떠한 식으로든 설명을 더하거나 개입하지 않는다.\n 그저 독자들을 밤과 어둠의 이미지로 안내할 뿐이다.\n \n\n패밀리레스토랑에서 혼자 책을 읽고 있는 마리에게 젊은 남자가 다가와 말을 건다.\n “혹시 아사이 에리 동생 아냐? 전에 우리 한 번 만났지?” 하룻밤 동안 마리는 다양한 사람들과 만나 대화를 주고받는다.\n 주로 잠을 빼앗긴 채 밤을 지새우고 있는 사람들이다.\n 밴드 주자, 중국인 창부, 러브호텔 스태프…… 마리는 왜 밤의 거리를 방황하는 걸까? 반대로 언니 에리는 왜 두 달째 깊은 잠에 빠져 있는 걸까? 밤을 걷는 사람들은 다들 어디에서 도망치고 싶은 걸까? 다양한 수수께끼를 머금은 찰나들이 스릴 있게 흐르고, 밤 11시 52분에 시작한 이야기는 익일 6시 52분을 기점으로 막을 내린다.\n"
# ✅ 입력 문장을 토큰화
input_ids = tokenizer(
    test_text, 
    return_tensors="pt", 
    max_length=1024, 
    truncation=True
    
    ).input_ids

# ✅ 모델로 요약 생성
summary_ids = model.generate(
    input_ids, 
    max_length=200, 
    num_beams=5, 
    early_stopping=True,
    repetition_penalty=1.5,
    length_penalty=1.5,
    temperature=0.8)
summary_text = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print("🔹 원문: ", test_text)
print("🔹 요약 결과: ", summary_text)


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


🔹 원문:  독자를 공모자로 끌어들인 채 관찰하는 자매의 하룻밤!

무라카미 하루키 데뷔 25주년 기념 장편소설 『애프터 다크』.
 1979년 《바람의 노래를 들어라》로 군조신인문학상을 수상하며 데뷔한 이후 독자들의 사랑을 받아온 무라카미 하루키.
 그가 등단 25주년을 맞는 해에 발표한 11번째 장편소설로, 발표 시기적으로는 《해변의 카프카》와 《1Q84》 사이에, 볼륨으로는 《국경의 남쪽, 태양의 서쪽》, 《스푸트니크의 연인》과 같은 장편소설 옆에 나란히 위치한다.


작품은 자정이 가까운 한밤에서부터 새날이 밝아오는 아침까지 일곱 시간 동안 벌어지는 백설 공주처럼 예쁜 언니 ‘에리’와 씩씩한 양치기 목동 같은 동생 ‘마리’, 두 자매의 이야기를 담고 있다.
 ‘우리’라고 명명된 카메라의 시선이 이야기를 주도한다.
 높은 곳에서 조감하는가 하면, 때로는 근접하여 클로즈업을 시도하며 영화의 장면들처럼 에리의 밤과 마리의 밤을 교대로 비추는 동안 작가는 어떠한 식으로든 설명을 더하거나 개입하지 않는다.
 그저 독자들을 밤과 어둠의 이미지로 안내할 뿐이다.
 

패밀리레스토랑에서 혼자 책을 읽고 있는 마리에게 젊은 남자가 다가와 말을 건다.
 “혹시 아사이 에리 동생 아냐? 전에 우리 한 번 만났지?” 하룻밤 동안 마리는 다양한 사람들과 만나 대화를 주고받는다.
 주로 잠을 빼앗긴 채 밤을 지새우고 있는 사람들이다.
 밴드 주자, 중국인 창부, 러브호텔 스태프…… 마리는 왜 밤의 거리를 방황하는 걸까? 반대로 언니 에리는 왜 두 달째 깊은 잠에 빠져 있는 걸까? 밤을 걷는 사람들은 다들 어디에서 도망치고 싶은 걸까? 다양한 수수께끼를 머금은 찰나들이 스릴 있게 흐르고, 밤 11시 52분에 시작한 이야기는 익일 6시 52분을 기점으로 막을 내린다.

🔹 요약 결과:  자정이 가까운 한밤에서부터 새날이 밝아오는 아침까지 일곱 시간 동안 벌어지는 백설 공주처럼 예쁜 언니 ‘에리’와 씩씩한 양치기 목동 같은 동생 ‘마리’의 이야기. 두 자매의 이야기를 그린 장편소설이다

In [ ]:
### 
## summary "" 로 작업용 키값을 설정했더니 줄거리가 없다고 인식하는 경우가 많았음